In [ ]:
#sudoku annealing
import sys, math, random
from copy import deepcopy

sys.path.append(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence')
from sudoku_parser import SudokuPuzzle

#helper functions"

def fixed_cells(puz):
    #bool grid for which cells we cant touch#
    return [[x!=0 for x in row] for row in puz.grid]

def block_cells(br, bc):
    for r in range(br*3, br*3+3):
        for c in range(bc*3, bc*3+3):
            yield r,c

def random_fill_blocks(puz, fixed):
    for br in range(3):
        for bc in range(3):
            had = {puz.grid[r][c] for r,c in block_cells(br,bc) if fixed[r][c]}
            empties = [(r,c) for r,c in block_cells(br,bc) if not fixed[r][c]]
            missing = [d for d in range(1,10) if d not in had]
            random.shuffle(missing)
            for (r,c), val in zip(empties, missing):
                puz.grid[r][c] = val

def rowcol_conflicts(puz):
    c = 0
    for r in range(9):
        c += 9 - len(set(puz.grid[r]))
    for cidx in range(9):
        col = [puz.grid[r][cidx] for r in range(9)]
        c += 9 - len(set(col))
    return c

def blk_valid(puz, br, bc):
    vals = [puz.grid[r][c] for r,c in block_cells(br,bc)]
    return set(vals) == set(range(1,10))

def solved(puz):
    for r in range(9):
        if len(set(puz.grid[r])) < 9: return False
    for c in range(9):
        if len(set(puz.grid[r][c] for r in range(9))) < 9: return False
    for br in range(3):
        for bc in range(3):
            if not blk_valid(puz,br,bc): return False
    return True

def get_bad_cells(puz):
    bad = set()
    for r in range(9):
        counts = {v:sum(1 for c in range(9) if puz.grid[r][c]==v) for v in range(1,10)}
        for c in range(9):
            if counts[puz.grid[r][c]]>1: bad.add((r,c))
    for c in range(9):
        counts = {v:sum(1 for r in range(9) if puz.grid[r][c]==v) for v in range(1,10)}
        for r in range(9):
            if counts[puz.grid[r][c]]>1: bad.add((r,c))
    return bad

def swap_try_best(puz, fixed, r,c,k=6):
    if fixed[r][c]: return None
    br, bc = r//3, c//3
    can = [(rr,cc) for rr,cc in block_cells(br,bc) if not fixed[rr][cc] and (rr,cc)!=(r,c)]
    if not can: return None
    sample = random.sample(can, min(k,len(can)))
    base = rowcol_conflicts(puz)
    best = None
    for rr,cc in sample:
        puz.grid[r][c], puz.grid[rr][cc] = puz.grid[rr][cc], puz.grid[r][c]
        newc = rowcol_conflicts(puz)
        puz.grid[r][c], puz.grid[rr][cc] = puz.grid[rr][cc], puz.grid[r][c]
        if best is None or newc < best[4]:
            best = (r,c,rr,cc,newc,base)
    return best

def do_swap(puz, move):
    r1,c1,r2,c2,_,_ = move
    puz.grid[r1][c1], puz.grid[r2][c2] = puz.grid[r2][c2], puz.grid[r1][c1]

def polish_blocks(puz,fixed,passes=5):
    for _ in range(passes):
        changed = False
        for br in range(3):
            for bc in range(3):
                vals = [puz.grid[r][c] for r,c in block_cells(br,bc)]
                pos = list(block_cells(br,bc))
                counts = {v: vals.count(v) for v in vals}
                dupes = [v for v,cnt in counts.items() if cnt>1]
                miss = [v for v in range(1,10) if v not in counts]
                if not dupes: continue
                for (r,c),v in zip(pos,vals):
                    if fixed[r][c]: continue
                    if v in dupes and miss:
                        puz.grid[r][c] = miss.pop(0)
                        dupes.remove(v)
                        changed=True
        if not changed: break

def anneal_sudoku(puz,fixed,maxsteps=250000,T0=4.0,cool=0.9992,log=5000):
    random_fill_blocks(puz,fixed)
    best_grid = deepcopy(puz.grid)
    best_cost = cur_cost = rowcol_conflicts(puz)
    T=T0
    for step in range(maxsteps):
        if cur_cost==0:
            polish_blocks(puz,fixed)
            if solved(puz):
                print(f"solved in {step} steps!")
                return True
        badcells = list(get_bad_cells(puz))
        if badcells:
            r,c = random.choice(badcells)
        else:
            br,bc = random.randint(0,2), random.randint(0,2)
            r,c = random.choice(list(block_cells(br,bc)))
        move = swap_try_best(puz,fixed,r,c)
        if move:
            _,_,_,_,new,base = move
            delta = new-base
            if delta<=0 or random.random()<math.exp(-delta/max(T,1e-9)):
                do_swap(puz,move)
                cur_cost=new
                if new<best_cost:
                    best_cost=new
                    best_grid=deepcopy(puz.grid)
        T*=cool
        if log and step%log==0:
            print(f"Step {step} | conflicts={cur_cost} | best={best_cost}")
    puz.grid = best_grid
    polish_blocks(puz,fixed)
    done = solved(puz)
    print("done!" if done else "ran out of steps")
    return done

def solve(file,attempts=5,seed=None):
    if seed: random.seed(seed)
    for i in range(attempts):
        print(f"\n--- try {i+1}")
        p = SudokuPuzzle(file)
        fixed = fixed_cells(p)
        print("start grid")
        p.display()
        if anneal_sudoku(p,fixed):
            print("final solved:")
            p.display()
            return
    print("all tries failed")

if __name__=="__main__":
    solve(r'C:\Users\gavin\OneDrive\Desktop\Artificial Intelligence\Med-P2.txt')
